In [9]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import glob
import pickle
import warnings
from scipy import misc
import scipy.ndimage
from skimage import feature
from PIL import Image
import os
import cv2
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Lambda, Input, Conv2DTranspose, Reshape, UpSampling2D
from keras.models import model_from_json, Model
from keras.callbacks import EarlyStopping
from keras import backend as K
import keras
from scipy.stats import norm

In [10]:
def get_train_hollow_data(procent, model_number):
    train_hollow_folder = "potato_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/train/hollow/"
    
    train_data_hollow = []
    
    for filename in os.listdir(train_hollow_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(train_hollow_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            train_data_hollow.append(img)
            
            
    np_train_data_hollow = np.array(train_data_hollow)
    
    train_data = np_train_data_hollow.reshape(np_train_data_hollow.shape[0], 128, 128, 1)
    
    return train_data

In [11]:
def get_train_metal_data(procent, model_number):
    train_metal_folder = "potato_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/train/metal/"
    
    train_data_metal = []
    
    for filename in os.listdir(train_metal_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(train_metal_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            train_data_metal.append(img)
            
            
    np_train_data_metal = np.array(train_data_metal)
    
    train_data = np_train_data_metal.reshape(np_train_data_metal.shape[0], 128, 128, 1)
    
    return train_data

In [12]:
def get_train_perfect_data(procent, model_number):
    train_perfect_folder = "potato_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/train/perfect/"
    
    train_data_perfect = []
    
    for filename in os.listdir(train_perfect_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(train_perfect_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            train_data_perfect.append(img)
            
            
    np_train_data_perfect = np.array(train_data_perfect)
    
    train_data = np_train_data_perfect.reshape(np_train_data_perfect.shape[0], 128, 128, 1)
    
    return train_data

In [13]:
# construct a custom layer to calculate the loss
class CustomVariationalLayer(keras.layers.Layer):

    def vae_loss(self, x, z_decoded, z_log_sigma, z_mu):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        # Reconstruction loss
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        # KL divergence
        kl_loss = -5e-4 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
        return K.mean(xent_loss + kl_loss)

    # adds the custom loss to the class
    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        z_log_sigma= inputs[2]
        z_mu = inputs[3]
        loss = self.vae_loss(x, z_decoded, z_log_sigma, z_mu)
        self.add_loss(loss, inputs=inputs)
        return x

In [14]:
def run_model(train_data, procent, model, potato_type, number_of_images):
    
    # Standard INFO
    img_shape = (128, 128, 1)
    batch_size = 16
    latent_dim = 100

    # Encoder
    input_img = Input(shape=img_shape)

    x = Conv2D(32, kernel_size=3,
                    padding='same', 
                    activation='relu',
                    strides=(2,2))(input_img)
    x = Conv2D(32, kernel_size=3,
                    padding='same', 
                    activation='relu',
                    strides=(2, 2))(x)

    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)

    # Two outputs, latent mean and (log)variance
    z_mu = Dense(latent_dim)(x)
    z_log_sigma = Dense(latent_dim)(x)
    
    
    
    
    def sampling(args):
        z_mu, z_log_sigma = args
        epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
                                  mean=0., stddev=1.)
        return z_mu + K.exp(z_log_sigma) * epsilon

    # sample vector from the latent distribution
    z = Lambda(sampling)([z_mu, z_log_sigma])
    
    
    # Decoder
    decoder_input = Input(K.int_shape(z)[1:])

    # Expand to 32 * 32 * 8 total pixels
    x = Dense(8192, activation='relu')(decoder_input)

    x = Reshape((32, 32, 8))(x)

    x = Conv2DTranspose(32, kernel_size=3,
                padding='same', 
                activation='relu',
                strides=(2, 2))(x)
    
    x = Conv2DTranspose(32, kernel_size=3,
                padding='same', 
                activation='relu',
                strides=(2, 2))(x)
    # reshape
    x = Conv2D(8, 3, padding='same', activation='relu')(x)
    x = Conv2D(1, 3,
                padding='same', 
                activation='sigmoid')(x)

    # decoder model statement
    decoder = Model(decoder_input, x)
    # apply the decoder to the sample from the latent distribution
    z_decoded = decoder(z)
    
    # apply the custom loss to the input images and the decoded latent distribution sample
    y = CustomVariationalLayer()([input_img, z_decoded, z_log_sigma, z_mu])
    
    
    vae = Model(input_img, y)
    vae.compile(optimizer='rmsprop', loss=None)
    vae.summary()
    decoder.summary()

    vae.fit(x=train_data, y=None, shuffle=True, epochs=8000 + (p * 50), batch_size=batch_size, verbose=0)
    
    
    
    data_path = "potato_dataset/data_" + str(procent) + "/model_" + str(model) + "/vae_4/" + potato_type + "/"
    n = 20
    image_size = 128
    figure = np.zeros((image_size * n, image_size * n))
        
    grid = norm.ppf(np.linspace(0.00, 1.00, 25))
    for i in range(number_of_images):
        sample = np.array([np.random.choice(grid, size=latent_dim, replace=True)])
        z_sample = np.tile(sample, batch_size).reshape(batch_size, latent_dim)
        x_decoded = decoder.predict(z_sample)
        final_image = x_decoded[0].reshape(image_size, image_size)
        plt.imsave(data_path + potato_type +"_vae_" + str(i) + ".jpg", final_image)
    
    K.clear_session()

    
    
    
    
    

In [ ]:
for p in range(10, 100, 10):
    start1 = time.time()
    for j in range(1, 11):
        start2 = time.time()
        hollow_data = get_train_hollow_data(p, j)
        run_model(hollow_data, p, j, "hollow", int(40 + p))
        end2 = time.time()
        print("Hollow Model {} with {} procent of the data took {} second".format(j, p, end2 - start2))
        
        start2 = time.time()
        metal_data = get_train_metal_data(p, j)
        run_model(metal_data, p, j, "metal", int(40 + p))
        end2 = time.time()
        print("metal Model {} with {} procent of the data took {} second".format(j, p, end2 - start2))
        
        start2 = time.time()
        perfect_data = get_train_perfect_data(p, j)
        run_model(perfect_data, p, j, "perfect", int(40 + p))
        end2 = time.time()
        print("perfect Model {} with {} procent of the data took {} second".format(j, p, end2 - start2))
    end1 = time.time()
    print("VAE for {} procent of data took {} second".format(p, end1 - start1))